# Module 1: Initial Setup

In [ ]:
# Setup
!pip install google-cloud-storage
from google.cloud import storage
import os
# Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()


MessageError: Error: credential propagation was unsuccessful

# Module 2: Trigger User Input- Upload Annual report of a company and feed Company name

In [ ]:
# Prerequisite
# Step 1- In Google Cloud Console, create a new project and link a billing account to it.
#Step 2- Navigate to the Cloud Storage section and click "Create Bucket".
#Step 3- Choose a unique bucket name, select a storage class and location, and click "Create".First, create a Vertex AI account and a project within the account. Then, enable billing for the project and create a storage bucket with input folders.
bucket_name = 'corp_spec_9696'
folder_name = 'input_documents'
# Input processing- Define Functions to upload file in gcs
from io import BytesIO
import smart_open
from tempfile import NamedTemporaryFile

def upload_file_to_gcs(file_path, bucket_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(file_path)
    print(f"File {file_path} uploaded to {destination_blob_name}.")

    # Construct and return the full GCS path
    gcs_path = f"gs://{bucket_name}/{destination_blob_name}"
    return gcs_path # Return the full GCS path
# User prompt trigger for Inputs
def prompt_user_for_input():
    """Prompts the user for input and returns the provided data."""
    print("Please provide input for the following fields:")
    annual_statement_i = input("Upload Annual Statement (pdf file): ")
    company_name_i = input("Enter Company Name: ")
    return annual_statement_i, company_name_i    #return annual_statement_i and company_name_i

# Call function to upload the input file in GCS
# Save input field (company name) as a variable


annual_statement_i, company_name_i = prompt_user_for_input()

annual_statement_path_i = None
annual_statement_path_i = upload_file_to_gcs(annual_statement_i, bucket_name, f"{folder_name}/annual_statement.{annual_statement_i.split('.')[-1]}")

company_name = company_name_i
print("Annual statement uploaded to:", annual_statement_path_i)
# Input processing- Define Functions to download file from gcs
!pip install langchain langchain-community PyPDF2



def get_pdf_temp_url(bucket_name, object_name):
    """Downloads a PDF from GCS and returns a temporary file-like URL.

    Args:
        bucket_name: The name of the GCS bucket.
        object_name: The name of the PDF object within the bucket.

    Returns:
        str: A temporary URL that can be used to access the PDF in memory.
    """

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(object_name)

    # Download directly into memory
    pdf_bytes = BytesIO()
    blob.download_to_file(pdf_bytes)
    pdf_bytes.seek(0)

    # Create a temporary file-like object
    with NamedTemporaryFile(suffix=".pdf", delete=False) as temp_file:
        temp_file.write(pdf_bytes.getvalue())
        temp_file_path = temp_file.name

    # Generate a file-like URL using smart_open
    pdf_url = smart_open.open(f"file://{temp_file_path}", "rb").name

    return pdf_url

# Example Usage:
from langchain.document_loaders import PyPDFLoader

bucket_name = "corp_spec_9696"
object_name = "input_documents/annual_statement.pdf"

pdf_url = get_pdf_temp_url(bucket_name, object_name)
print(pdf_url)


Please provide input for the following fields:
Upload Annual Statement (pdf file): /content/2024_Annual_Reportmicro.pdf
Enter Company Name: microsoft
File /content/2024_Annual_Reportmicro.pdf uploaded to input_documents/annual_statement.pdf.
Annual statement uploaded to: gs://corp_spec_9696/input_documents/annual_statement.pdf
/tmp/tmpam9pmc8u.pdf


# Module 3: Initialize LLM model- gemini-1.5-pro-001

In [ ]:
# initiate setup and libraries
import base64
import vertexai
import json
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from vertexai.preview.language_models import TextGenerationModel
PROJECT_ID = "ai-hospital-agents-9696"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)
MODEL_ID = "gemini-1.5-pro-001"  # @param {type:"string"}
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


# Module 4: Extract general details about company- Company website, Relevant Industry and Products


In [ ]:
# Prompt for extracting relevant information
prompt = f"""for this company: {company_name}, generate following fields- Company website, Relevant Industries, List of close competitors, Products/services"""
print(prompt)
# Define function to Extract data and Generate response
def generate(prompt):
    PROJECT_ID = "ai-hospital-agents-9696"  # @param {type:"string"}

    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel(
        MODEL_ID,
        system_instruction=["""you have a company name as input, generate only the data asked in prompt in this format-
        field x: generated data relevant to field x
        ; field y: generated data relevant to field y"""]
    )

    responses = model.generate_content(
        [prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    full_response_text = ""
    for response in responses:
        full_response_text += response.text
    # Return the full response text
    return full_response_text

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

# Generate responses and store the result
saved_response_prep = generate(prompt)

# Print the saved response
print(saved_response_prep)
# Split the response to save as individual variables

fields = saved_response_prep.split(';')

# Initialize variables to store the extracted values
company_website = "N/A"
relevant_industry = "N/A"
competitors ="N/A"
products = "N/A"

# Iterate over the fields and extract key-value pairs
for field in fields:
    if ':' in field:
        key, value = field.split(':', 1)  # Split only at the first occurrence of ':'
        key = key.strip()
        value = value.strip()

        if key == "Company website":
            company_website = value
        elif key == "Relevant Industries":
            relevant_industry = value
        elif key == "List of close competitors":
            competitors = value
        elif key == "Products/services":
            products = value

# Loop through the variable names and values to print them
for var_name in [
    "company_website", "relevant_industry", "competitors", "products"
]:
    value = globals()[var_name]  # Get the value of the variable using its name
    print(f"{var_name.replace('_', ' ')}: {value}")


for this company: microsoft, generate following fields- Company website, Relevant Industries, List of close competitors, Products/services
Company website: www.microsoft.com
; Relevant Industries: Software Development, Cloud Computing, Artificial Intelligence, Video Games, Consumer Electronics, Hardware 
; List of close competitors: Apple, Google, Amazon, Meta (Facebook), Oracle, Salesforce
; Products/services: Windows operating system, Azure cloud services, Office 365 productivity suite, Xbox gaming consoles and services, Surface hardware, LinkedIn professional networking 

company website: www.microsoft.com
relevant industry: Software Development, Cloud Computing, Artificial Intelligence, Video Games, Consumer Electronics, Hardware
competitors: Apple, Google, Amazon, Meta (Facebook), Oracle, Salesforce
products: Windows operating system, Azure cloud services, Office 365 productivity suite, Xbox gaming consoles and services, Surface hardware, LinkedIn professional networking


# Module 5: Call Custom Search Engine to extract latest News and trends on Company and Industry and Sentiment Analysis

In [ ]:

# First, install the required libraries
!pip install google-api-python-client

# Import necessary libraries
from googleapiclient.discovery import build

# Define your API key and Search Engine ID
from google.colab import userdata

api_key = userdata.get('APIkeyCSE')
cse_id = userdata.get('CSEid')
# Function to perform Google search on Relevant Industry News and trends
def google_search(query, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, **kwargs).execute()
    return res

# Sample query
query_ind = f"""Industry report and trends for: {relevant_industry}"""

# Perform the search
results_ind = google_search(query_ind, api_key, cse_id)

# Display results
for item in results_ind.get('items', []):
    print(f"Title: {item['title']}")
    print(f"Snippet: {item['snippet']}")
    print(f"Link: {item['link']}\n")

print(results_ind)
#print(results)
# Extract title and snippets from Industry News response
industry_title = []
industry_snippet = []
for item in results_ind.get('items', []):
    industry_title.append(f"Title: {item['title']}")
#    industry_snippet.append(f"Snippet: {item['snippet']}")
    industry_snippet.append(f"{item['snippet']}")
print(f"industry news titles are {industry_title}")
print(f"industry news snippets are {industry_snippet}")

# Sentiment analysis
from textblob import TextBlob

# Extract sentiment scores
sentiment_scores = []
for item in results_ind.get('items', []):
    analysis = TextBlob(item['snippet'])
    sentiment_scores.append(analysis.sentiment.polarity)  # -1 (negative) to 1 (positive)

average_sentiment = sum(sentiment_scores) / len(sentiment_scores)
print("Average News Sentiment:", average_sentiment)

def analyze_sentiment(score):
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"

sentiment = analyze_sentiment(average_sentiment)
print("Overall Sentiment:", sentiment)

sentiment_category = analyze_sentiment(average_sentiment)
print(f"Average News Sentiment: {average_sentiment:.3f} ({sentiment_category})")

if sentiment_category == "Positive":
    print("Significance: This suggests a generally favorable outlook for  based on recent news coverage. Investors may interpret this as a sign of positive momentum.")
elif sentiment_category == "Negative":
    print("Significance: This indicates a negative sentiment towards in recent news. Investors may interpret this as a warning sign or a potential for downward pressure on the stock price.")
else:
    print("Significance: News sentiment is neutral, suggesting a lack of strong positive or negative opinions about in recent coverage. Investors may look for other indicators to gauge market sentiment.")
# Function to perform Google search on Company related News and trends
def google_search(query, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, **kwargs).execute()
    return res

# Sample query
query = f"""Latest News and trends of: {company_name}"""

# Perform the search
results = google_search(query, api_key, cse_id)

# Display results
for item in results.get('items', []):
    print(f"Title: {item['title']}")
    print(f"Snippet: {item['snippet']}")
    print(f"Link: {item['link']}\n")
#print(results)
# Extract title and snippets from Company News response
company_title = []
company_snippet = []
for item in results.get('items', []):
    company_title.append(f"Title: {item['title']}")
#    company_snippet.append(f"Snippet: {item['snippet']}")
    company_snippet.append(f"{item['snippet']}")
print(f"company news titles are {company_title}")
print(f"company news snippets are {company_snippet}")


Title: Artificial Intelligence (AI) In Video Games Market Report 2024, Share ...
Snippet: 1) By Component: Hardware, Software, Service · 2) By Technology: Machine Learning, Natural Language Processing, Computer Vision, Gesture Control, Other ...
Link: https://www.thebusinessresearchcompany.com/report/artificial-intelligence-ai-in-video-games-global-market-report

Title: McKinsey Technology Trends Outlook 2022
Snippet: Aug 1, 2022 ... Increase in products and services the industry can offer, spanning cloud ... Source: Gartner; “Software development AI market overview,” ...
Link: https://www.mckinsey.com/~/media/mckinsey/business%20functions/mckinsey%20digital/our%20insights/the%20top%20trends%20in%20tech%202022/mckinsey-tech-trends-outlook-2022-full-report.pdf

Title: Microsoft 2023 Annual Report
Snippet: ... software development tools, and video games. We also design and sell devices ... Nuance is a cloud and artificial intelligence (“AI”) software provider ...
Link: https://www.micros

# Module 6: Call Alpha Vantage API and extract market data available

In [ ]:


!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
# Feed company name to extract ticker symbol (a unique grouping of letters that identifies a specific company, e.g. AAPL for Apple)
# Enter api_key for Alpha Vantage API key
api_key = 'APIkeyAV'
def find_ticker(company_name):
    ts = TimeSeries(key=api_key)
    data, meta_data = ts.get_symbol_search(company_name)
    if not data.empty:  # Check if the DataFrame is empty
        return data.iloc[0]['1. symbol'] # Access the first row and '1. symbol' column
    else:
        return "Ticker not found."

ticker = find_ticker(company_name)

print(f"The ticker symbol for {company_name} is {ticker}.")
# ticker = 'JIOFIN'
print(ticker)
import requests
# Extract Company Overview
overview_data = requests.get(
    f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={api_key}"
).json()
if overview_data:
    print("Company Overview:")
    print("\n".join(f"{k}: {v}" for k, v in overview_data.items()))
# Extract Intraday data
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=5min&apikey={api_key}'
r = requests.get(url)
data = r.json()
print(data)
# Time series Intraday data plotting
# import requests
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates

# # Alpha Vantage API call (assuming ticker symbol and 5-minute interval)
# Data Extraction and Transformation
time_series_data = data['Time Series (5min)']
df = pd.DataFrame.from_dict(time_series_data, orient='index')
df = df.astype(float)
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(df['4. close'], marker='o', linestyle='-')
plt.title(f'{ticker} Intraday 5-minute Close Prices')
plt.xlabel('Time (US/Eastern)')
plt.ylabel('Close Price')
plt.grid(axis='y')


# Create a new DataFrame with hourly data (for tick locations)
hourly_df = df.resample('H').last()  # Resample to hourly intervals

# Format x-axis labels to display only hours
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H'))
plt.xticks(hourly_df.index)  # Set the x ticks to be at the hour positions

plt.tight_layout()
plt.show()


The ticker symbol for microsoft is MSF0.FRK.
MSF0.FRK
{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.'}


KeyError: 'Time Series (5min)'

# Module 7: Breakdown unstructured pdf into Vector Embeddings

In [ ]:


# Initiating libraries for Langchain
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install --upgrade chromadb
!pip install tiktoken
!pip install sentence-transformers
!pip install google.generativeai
!pip install langchain-google-genai
from langchain.document_loaders import PyPDFLoader
from langchain import ConversationChain, LLMChain
from langchain.memory import SimpleMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from sentence_transformers import SentenceTransformer
from chromadb.api.types import Documents, Embeddings

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import chromadb
import numpy as np
from google.colab import userdata
# Set up the environment variable for Google API Key (Store in secrets)

import os
os.environ["GOOGLE_API_KEY"] = userdata.get('gcloudAPIKey')
# Building up the additional context from Modules 4, 5 and 6
add_context = f"Relevant industry is {relevant_industry}, \n Industry news sentiment is {sentiment}, \n company offerings are {products}, \n industry news snippets are {industry_snippet},\n Intraday company data is {data}"
print(add_context)
# PDF processing and adding additional context

# Step 1: Load the PDF document

pdf_loader = PyPDFLoader(pdf_url)

documents = pdf_loader.load()
total_pages = len(documents)
print(f"PDF loaded with {total_pages} pages.")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10
)

all_text = ""
for document in documents:
    all_text += document.page_content + "\n\n"

# Add the additional context only once to the combined text
all_text += add_context
texts = text_splitter.split_text(all_text)

print(f"Total chunks created: {len(texts)}")

# Step 3: Initialize Sentence Transformer model
model_batch = SentenceTransformer('all-MiniLM-L6-v2')
# Step 4: Process in batches
batch_size = 100
embeddings = []
# metadata = []
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    print(f"Encoding batch starting at index {i}")
    batch_embeddings = model_batch.encode(batch_texts, show_progress_bar=True)
    embeddings.append(batch_embeddings)
# Creating and adding Embeddings in collection
# Step 5: Concatenate all batch embeddings
embeddings_np = np.concatenate(embeddings, axis=0)


# Step 6: Initialize Chroma client
client1 = chromadb.Client()

# Step 8: Create a Chroma collection
# Delete the collection if it exists
client1.delete_collection("my_collection")
collection = client1.create_collection("my_collection")

# Step 9: Add embeddings to the collection
collection.add(
    embeddings=embeddings_np.tolist(),
    documents=texts,
    ids=[str(i) for i in range(len(embeddings_np))],
    # metadata= metadata
)
# Get a sample of documents from the collection
sample_docs = collection.get(limit=5)

# Print the sample documents
print(sample_docs)


  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Succe

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding batch starting at index 100


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding batch starting at index 200


...

Encoding batch starting at index 4500


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding batch starting at index 4600


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

{'ids': ['0', '1', '2', '3', '4'], 'embeddings': None, 'documents': ['1 \nDear shareholders, colleagues, customers, and partners:', 'Fiscal year 2024 was a pivotal year for Microsoft. We entered our 50th year as a company and the', 'and the second year of the AI', 'platform shift. With these milestones, I’ve found myself reflecting on how Microsoft has remained a', 'a consequential company'], 'uris': None, 'data': None, 'metadatas': [None, None, None, None, None], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


# Module 8: Feed embeddings to gemini-1.5.pro model for insights

In [ ]:

!pip install langchain huggingface_hub transformers



from langchain.embeddings import HuggingFaceEmbeddings

model_embed = "sentence-transformers/all-MiniLM-L6-v2"

embeddings1 = HuggingFaceEmbeddings(
    model_name=model_embed
)# Feed embeddings to LLM along with query

llm = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=1)

vectorstore = Chroma(
    client=client1,  # Use the same client from the Module 7
    collection_name="my_collection",
    embedding_function=embeddings1
)

# Create the retriever from the Langchain Chroma vectorstore
retriever = vectorstore.as_retriever(search_type="similarity")

def create_retrieval_chain(retriever, combine_docs_chain):
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
    )


# Create the retrieval chain
template = """You are a very professional document summarization and analysis specialist.
  Understand the factors for company profitability in context of the annual statement attached in pdf_file,
  respond as a finance expert with useful insights, provide simple tabular presentation, if needed.
  consider user a novice, the response should be very intuitive and easy to understand.
context: {context}
input: {input}
answer:
"""

prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
response=retrieval_chain.invoke({"query":"industry sentiment? market data? net profit?"})   #, what should be the company strategy for new products and services?"})

#Print the answer to the question
print(response["result"])



Industry sentiment is Negative. I don't have enough information to answer the other questions. 



# Module 9: Build a chatbot to respond in live chat session

In [1]:


# Create a Chatbot
conversation_history = []  # List to store conversation history

def prompt_chatbot_query():
    while True:
        user_input = input("Enter query: ")
        if user_input.lower() == "exit":
            break

        # Append the user's query to conversation history
        conversation_history.append({"query": user_input})

        # Create context including previous conversation
        history_str = "\n".join([f"User: {turn['query']}\nBot: {turn.get('result', '')}" for turn in conversation_history])
        # context = f"You are a document summarizer, based on the {history_str}\nCurrent query: {user_input}\n {add_context}"
        context = f"""You are a very professional document summarization and expert financial analyst.
       Understand the factors and interdependencies, and respond as an expert with useful insights, provide simple tabular presentation, if needed.

         \n answer concisely \n
        {history_str}\nCurrent query: {user_input}"""


        # Invoke retrieval chain with the updated context
        response = retrieval_chain.invoke({"query": context, "input": user_input})

        # Append the model's response to the conversation history
        conversation_history[-1]["result"] = response["result"]

        print(response["result"])

prompt_chatbot_query()

Enter query: give insights


NameError: name 'retrieval_chain' is not defined